In [7]:
from dotenv import load_dotenv
import os
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("YoungWon")

LangSmith 추적을 시작합니다.
[프로젝트명]
YoungWon


### 프롬프트 템플릿의 활용

In [2]:
from langchain_teddynote.messages import stream_response    # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [3]:
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [ ]:
prompt = prompt_template.format(country="미국")
prompt

### Chain 생성

#### invoke() 호출 
- python 딕셔너리 형태로 입력값을 전달
- invoke() 함수 호출 시, 입력값을 전달

In [9]:
from langchain_openai import ChatOpenAI

openai_api_key = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(
    api_key=openai_api_key,
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

prompt = PromptTemplate.from_template("{topic}에 대해 쉽게 설명해주세요")

chain = prompt | model

input = {"topic": "인공지능 모델의 학습 원리"}

chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고, 이 데이터를 분석하여 패턴을 찾아내는 과정입니다. 이 과정은 크게 입력층, 은닉층, 출력층으로 구성된 인공신경망을 사용하여 이루어집니다.\n\n먼저, 입력층에는 데이터가 입력되고, 은닉층에서는 입력된 데이터를 분석하여 패턴을 찾아냅니다. 이후, 출력층에서는 은닉층에서 찾아낸 패턴을 바탕으로 결과를 출력합니다.\n\n이러한 과정을 반복하면 모델은 점차적으로 데이터의 패턴을 학습하고, 새로운 데이터가 입력되었을 때 정확한 결과를 예측할 수 있게 됩니다. 이렇게 학습된 모델은 다양한 분야에서 활용되어 사람들의 일상생활을 편리하게 해주는 다양한 서비스를 제공하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 300, 'prompt_tokens': 32, 'total_tokens': 332, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-25aefc79-bfe7-473f-bdda-d504a9293a06-0', usage_metadata={'input_tokens': 32, 'output_tokens': 300, 'total_tokens': 332})

In [10]:
answer = chain.stream(input)
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고 이를 처리하여 원하는 결과를 출력하는 과정입니다. 이 과정은 크게 입력층, 은닉층, 출력층으로 구성된 인공신경망을 사용하여 이루어집니다.

먼저, 모델은 입력층에서 데이터를 받아들입니다. 이 데이터는 숫자로 이루어진 벡터 형태로 표현되며, 각각의 숫자는 모델이 학습할 특징을 나타냅니다. 

다음으로, 입력된 데이터는 은닉층을 거쳐 처리됩니다. 은닉층은 입력층과 출력층 사이에 위치하며, 입력된 데이터에 가중치를 곱하고 편향을 더하여 새로운 값을 계산합니다. 이를 통해 모델은 입력된 데이터의 패턴을 학습하고 추상화된 특징을 추출합니다.

마지막으로, 출력층에서는 은닉층을 거친 데이터를 최종적으로 처리하여 원하는 결과를 출력합니다. 이 결과는 모델이 학습한 패턴을 바탕으로 예측된 값이며, 이를 토대로 모델의 성능을 평가하고 개선하는 과정을 반복합니다.

이렇게 모델은 입력된 데이터와 원하는 결과를 비교하면서 학습을 진행하고, 가중치와 편향을 조정하여 최적의 결과를 도출합니다. 이러한 과정을 통해 인공지능 모델은 주어진 데이터를 학습하고 예측하는 능력을 향상시킵니다.

### 출력파서(Output Parser)

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

input = {"topic" : "인공지능 모델의 학습 원리"}

chain.invoke(input)

answer = chain.stream(input)
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고 이를 분석하여 패턴을 찾아내는 과정입니다. 모델은 입력된 데이터를 기반으로 가중치를 조정하고 최적의 결정을 내리는 방법을 학습합니다. 이를 통해 모델은 주어진 데이터에 대한 패턴을 학습하고, 새로운 데이터에 대해 예측이나 분류를 수행할 수 있게 됩니다. 학습 과정에서 모델은 오차를 최소화하도록 가중치를 조정하며, 이를 통해 데이터에 대한 정확한 예측을 할 수 있게 됩니다.

In [14]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황: 
{question}

Format:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용해 프롬프트를 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗 모델 초기화
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

chain = prompt | model | output_parser

answer = chain.stream({"question":"저는 식당에서 음식을 주문하고 싶어요"})
stream_response(answer)

영어 회화:
- Hello, I'd like to order now, please.
- Could I get the grilled salmon with a side of vegetables?
- And for my drink, I'll have a glass of water, thank you.
- That's all for now. How long do you think it will take?

한글 해석:
- 안녕하세요, 지금 주문하고 싶어요.
- 구운 연어와 야채 사이드를 주문할 수 있을까요?
- 음료는 물 한 잔으로 할게요, 감사합니다.
- 이게 다예요. 얼마나 걸릴까요?

In [15]:
answer = chain.stream({"question":"미국 입국 심사"})
stream_response(answer)

영어 회화:
- Immigration Officer: "Good morning, may I see your passport and any customs declaration form?"
- Traveler: "Good morning! Here is my passport and the form."
- Immigration Officer: "Thank you. What is the purpose of your visit?"
- Traveler: "I'm here for tourism. I plan to visit several landmarks and enjoy some local cuisine."
- Immigration Officer: "Sounds great. How long do you plan to stay?"
- Traveler: "I will be staying for two weeks."
- Immigration Officer: "Where will you be staying during your visit?"
- Traveler: "I have reservations at a hotel in downtown Manhattan."
- Immigration Officer: "Okay, everything seems in order. Welcome to the United States. Enjoy your stay!"
- Traveler: "Thank you very much!"

한글 해석:
- 입국 심사관: "좋은 아침입니다, 여권과 세관 신고서를 볼 수 있을까요?"
- 여행자: "좋은 아침입니다! 여기 여권과 신고서가 있습니다."
- 입국 심사관: "감사합니다. 방문 목적이 무엇인가요?"
- 여행자: "관광을 위해 왔습니다. 몇몇 명소를 방문하고 현지 음식을 즐길 계획입니다."
- 입국 심사관: "좋네요. 얼마 동안 머무를 예정인가요?"
- 여행자: "2주 동안 머무를 예정입니다."
- 입국 심사관: "방문 동안 어디에 머무를 예정인가요?"
- 여